In [1]:
import numpy as np
import pandas as pd
import torch
import os
import zarr
import anndata
import h5py

from dredFISH.Analysis import basicu

In [2]:
f = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/top_recon_genes_dpnmf_nn_n140.csv'
genesp = pd.read_csv(f, header=None)[0].values
genesp, genesp.shape

(array(['Stmn3', 'Neurod6', 'Bex2', 'Lhx6', 'Dlx1', 'Tmem59l', 'Neurod2',
        'Satb2', 'Ptk2b', 'Rab3a', 'Uchl1', 'Gng3', 'Dnm1', 'Fezf2',
        'Dlx5', 'Lingo1', 'Nfix', 'Zfp36l1', 'Fxyd7', 'Scg5', 'Nxph3',
        'Crym', 'Arx', 'Eno2', 'Nxph4', 'Ifitm3', 'Fcrls', 'Zcchc12',
        'Rab3b', 'Acot7', 'Cpne4', 'Kcnv1', 'Prkar1b', 'Atp1a1', 'Cox6a2',
        'Slc30a3', 'Igfbp6', 'Tagln3', 'Elmod1', 'Pnoc', 'Nsg1', 'Mllt11',
        'Chgb', 'Atp2b4', 'Btbd11', 'Arpp19', 'Ttc9b', 'Sez6', 'Hs3st2',
        'Nefl', 'Snap47', 'Hprt', 'Tesc', 'Pglyrp1', 'Atp6v1b2', 'Tac2',
        'Cd63', 'Hsp90aa1', 'Syn1', 'Gli3', 'Adora1', 'Lgals1', 'Got1',
        'Arhgap25', 'Serp2', 'Syt5', 'Sox17', 'Stx1a', 'Nr2f2', 'Dkkl1',
        'Hpcal4', 'Prkcg', 'Rasgef1b', 'Dkk3', 'Cckbr', 'Bex1', 'Ngef',
        'Tceal3', 'Tspan13', 'Kcnip3', 'Dusp14', 'Fstl4', 'S100a16', 'Cfh',
        'Osbpl1a', 'Anxa3', 'Itpr1', 'Krt12', 'Pdzrn3', 'Gabrg2', 'Olfm2',
        'Atp6v1c1', 'Pamr1', 'Atp5b', 'Igsf21', 'Kct

In [3]:
dat_dir = '/bigstore/GeneralStorage/fangming/projects/dredfish/data/'
zarr_file = os.path.join(dat_dir, 'rna', 'scrna_10x_ctxhippo_a_exon_count_matrix_v2.zarr')
f = '/bigstore/binfo/mouse/Brain/Sequencing/Allen_10X_CtxHippo/source/expression_matrix.hdf5'
print(zarr_file)

z1 = zarr.open(zarr_file, mode='r') 
print(z1, list(z1.keys()))
print(z1['counts'].shape)


with h5py.File(f, 'r') as fh:
    # genes
    genes = fh['data']['gene'][()]
    genes = np.array([gene.decode() for gene in genes])
    print(genes.shape)

# get NN genes
nn_genes = genes
nn_genes

/bigstore/GeneralStorage/fangming/projects/dredfish/data/rna/scrna_10x_ctxhippo_a_exon_count_matrix_v2.zarr
<zarr.hierarchy.Group '/' read-only> ['counts', 'l1_cat', 'l1_code', 'l2_cat', 'l2_code', 'l3_cat', 'l3_code', 'l5_cat', 'l5_code', 'num_probe_limit']
(1169213, 31053)
(31053,)


array(['Xkr4', 'Gm1992', 'Gm37381', ..., 'CAAA01118383.1', 'Vmn2r122',
       'CAAA01147332.1'], dtype='<U29')

In [4]:
# sub select the nn_genes
selected_genes_idx = basicu.get_index_from_array(nn_genes, genesp)
fout = os.path.join(dat_dir, 'rna', 'gidx_sub140_tenx_v1.pt')
torch.save(torch.tensor(selected_genes_idx, dtype=int), fout)

gsub_idx = torch.load(fout)
gsub_idx, gsub_idx.shape

(tensor([ 4078, 10691,  4948,  2219,  2448, 16721, 22191,   352, 18710, 16739,
          8890, 30364,  2142, 17678, 10178, 19806, 16927, 24812, 12703,  3061,
         22102, 13980,  4696, 11557, 15874, 14354,  5698,  4335,  7423,  8237,
         20375, 25779,  9920,  6093, 14166,  8608, 26561, 27180, 19766, 18697,
          8738,  5956,  3373,  1045, 15449, 20067, 12538, 21815, 14001, 18734,
         21379,  4460,  9520, 12235, 16676, 15883, 15948, 25201,  4251, 22917,
          1056, 26100, 30773, 11172, 18845, 11962,     4,  9789, 13202, 12969,
          7633, 11916,  9184, 13891, 13749,  4953,   698,  4959, 24488,  3262,
         21950, 21244,  5828,  1141, 29599,  9160, 11337, 22235, 11317, 21080,
         19203, 25738,  2927, 15899,  8001, 18838, 30748,  6268, 22710, 23219,
          8210, 10576,  3141,  2484, 18713,   557, 14377, 18778, 27848, 26901,
         27973,  6585, 24649, 15715,  8709, 25546,  6841, 12612,  9363, 22192,
         29690,  8949,  6022,  9509, 20454, 30304,  